In [ ]:
import ffmpeg
import subprocess
from pathlib import Path

### Flipping a Video

In [ ]:
file = ffmpeg.input("Mein Pocha Chaand Se.mp4",f='mp4')
horizontalvideo = ffmpeg.hflip(file)
audio = file.audio
video_bitrate_value = 1200 # In mbps
audio_bitrate_value = 192 # In kbps
Output = ffmpeg.output(horizontalvideo,audio,"filpedvideo.mp4",format='mp4',video_bitrate=video_bitrate_value)


In [13]:
ffmpeg.run(Output)

(None, None)

### Cropping a Video and compressing 

In [9]:
file = ffmpeg.input("Test-Video.mov")
croppedvideo = ffmpeg.crop(
    x='(iw-ow)/2',
    y=0,
    width='ih*(9/16)',
    height='ih',
    stream=file.video )
audio = file.audio
video_bitrate_value = "1200k" # In kbps
audio_bitrate_value = "192k" # In kbps
Output = ffmpeg.output(croppedvideo,audio,"croppedvideo.mp4",video_bitrate=video_bitrate_value,audio_bitrate=audio_bitrate_value)
Output.run() # Output the Video


(None, None)

FFmpeg in the sermon_splitter

In [ ]:

class FfmpegManager:
    def __init__(self, ffmpeg_path: str = FFMPEG):
        self.ffmpeg_path = ffmpeg_path

    def run_command(self, args: list):
        """Executes an FFmpeg command and raises an exception if it fails."""
        proc = subprocess.run([self.ffmpeg_path, *args], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if proc.returncode != 0:
            raise RuntimeError(f"ffmpeg failed:\n{proc.stderr}")
        return proc

In [ ]:
class VideoProcessor:
    def __init__(self, ffmpeg_manager: FfmpegManager):
        self.ffmpeg = ffmpeg_manager
        
    def cut_video_clip(self, source_mp4: Path, start_time: str, end_time: str, out_file: Path):
            """Accurate re-encode cut with audio."""
            args = [
                "-y", #Overrite without asking
                "-i", str(source_mp4), #input Source 
                "-ss", start_time, "-to", end_time, #start and stop
                "-map", "0:v:0", "-map", "0:a:0?",  #Maps the starting of Video and audio (Inputs)
                "-c:v", "libx264", "-preset", "veryfast", "-crf", "18",  # Video Rendering Attributes
                "-pix_fmt", "yuv420p", # Video Rendering Attributes
                "-c:a", "aac", "-b:a", "192k", # Audio Rendering Attributes
                "-movflags", "+faststart", # moves metadata to the beginningof the file
                str(out_file) #Output File
            ]
            self.ffmpeg.run_command(args) 
            
            
    def extract_audio_to_wav(self, input_mp4: Path, wav_path: Path, sample_rate=16000):
        """Extracts audio from a video file to a mono 16kHz WAV file."""
        args = [
            "-y", #Overwrite without asking 
            "-i", str(input_mp4), #Input Source 
            "-vn", # Excludes video while rendering
            "-acodec", "pcm_s16le", # Audio Attribute (refering the codec to use)
            "-ac", "1", #No. of Audio Channels
            "-ar", str(sample_rate), #bitrate/Sample Rate of the audio
            str(wav_path) # Output
        ]
        self.ffmpeg.run_command(args) 